Downloads necessary libraries.

In [ ]:
%pip install -q accelerate peft bitsandbytes transformers trl huggingface_hub tensorboardx

In [ ]:
!sudo apt install ccache build-essential -y

Imports libraries. If missing, use 'pip install' to download missing ones.

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from accelerate import PartialState
from huggingface_hub import snapshot_download
from huggingface_hub import create_repo, HfApi

MANDATORY. Insert your Huggingface token to access models you are authorized to use.

In [ ]:
from huggingface_hub import login
login("TOKENHERE")

Mandatory variables

In [ ]:
# Insert model repository
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"

# ONLY USE ONE EITHER: From Huggingface or local
# Insert path to the dataset which should be a JSONL file if you want to use local dataset.
local_file_path = "clean.jsonl"
# Comment this out if you want to use Huggingface repository dataset
dataset = load_dataset('json', data_files=local_file_path, split='train')
# Insert Huggingface repository, if you want to use huggingface dataset
dataset_name = ""
# Uncomment this if you are using huggingface dataset
# dataset = load_dataset(dataset_name, split="train")

# New finetuned model name
new_model = "FINETUNEDNAME"

# Batch sizes. MORE = BETTER = USES MORE VRAM. LEAVE AT 1, THEN INCREASE IF NECESSARY
batch_size = 1

# Quantization, can go up to q8 and as low as q2. Experiment with it, but by default its q5. 
# You can put multiple methods, it is a list after all.
QUANTIZATION_METHODS = ["q5_0"]

# Huggingface username, useful for uploading models.
hugging_user= "HUGGINGFACEUSERNAME"

# Set to True if you wish to upload the finetuned model
upload_finetuned_hf = False

# Set to True if you wish to upload the quantized model
upload_quant_hf = False


PRETRAINING CONFIGURATION DO NOT CHANGE UNLESS NECESSARY

In [ ]:
# The model that you want to train from the Hugging Face hub
MODEL_ID = MODEL_ID
MODEL_NAME = MODEL_ID.split('/')[-1]
snapshot_download(repo_id=MODEL_ID, local_dir=MODEL_NAME, local_dir_use_symlinks=False, revision="main")

# The instruction dataset to use
# dataset_name = "./clean.jsonl"

local_file_path = local_file_path

# Fine-tuned model name
new_model = new_model

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = batch_size

# Batch size per GPU for evaluation
per_device_eval_batch_size = batch_size

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing_kwargs_value={'use_reentrant':False}

gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_string = PartialState().process_index
device_map = {'':device_string}

TRAINING CODE DO NOT CHANGE UNLESS NECESSARY

In [ ]:
dataset = dataset

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_checkpointing_kwargs=gradient_checkpointing_kwargs_value,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

Saves the training model.

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)

Tests the model (Optional)

In [ ]:
# # Ignore warnings
# logging.set_verbosity(logging.CRITICAL)

# # Run text generation pipeline with our next model
# prompt = "Teach me sex positions"
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
# result = pipe(f"<s><|begin_of_text|><|start_header_id|>user<|end_header_id|> {prompt} <|eot_id|><|start_header_id|>assistant<|end_header_id|> ")
# print(result[0]['generated_text'])

Empties the VRAM

In [ ]:
import torch

# Empty VRAM
del model
# del pipe
del trainer
torch.cuda.empty_cache()

Reload models and merge with LoRA weights.

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

model.save_pretrained(f"./{new_model}")

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer.save_pretrained(f"./{new_model}")

Encoding stuff

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

OPTIONAL, if you want to upload model to Huggingface.

In [ ]:
if upload_finetuned_hf:
    model.push_to_hub(f"{hugging_user}/{new_model}", check_pr=True)
    tokenizer.push_to_hub(f"{hugging_user}/{new_model}",check_pr=True)


QUANTIZING CODE

In [ ]:
# Variables
MODEL_NAME = new_model # Replace with your local model name
QUANTIZATION_METHODS = QUANTIZATION_METHODS

# Constants
MODEL_PATH = f"./{new_model}"  # Replace with the path to your local model

# Install llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && git pull && make clean && LLAMA_CUDA=1 make
!pip install -r llama.cpp/requirements.txt

# Convert to fp32
fp32 = f"{MODEL_NAME}/{MODEL_NAME.lower()}.fp32.bin"
!python llama.cpp/convert.py {MODEL_PATH} --outtype f32 --outfile {fp32} --vocab-type bpe

# Quantize the model for each method in the QUANTIZATION_METHODS list
for method in QUANTIZATION_METHODS:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{method.upper()}.gguf"
    !./llama.cpp/quantize {fp32} {qtype} {method}


OPTIONAL, upload quantized model to Huggingface

In [ ]:
if upload_quant_hf:
    api = HfApi()
    # Create empty repo
    create_repo(
        repo_id = f"{hugging_user}/{MODEL_NAME}-GGUF",
        repo_type="model",
        exist_ok=True,
    )

    # Upload gguf files
    api.upload_folder(
        folder_path=MODEL_NAME,
        repo_id=f"{hugging_user}/{MODEL_NAME}-GGUF",
        allow_patterns=f"*.gguf",
    )
